# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.auto import tqdm
import os

## For Google Colab Users

This cell is for mounting your Google Drive to the Colab Notebook. If you are not using Google Colab, you can skip this cell

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Check for GPU in mac
# device = "mps" if torch.backends.mps.is_available() else "cpu"

device

'cpu'

# Data 

## Transforming Data

In [4]:
data_transforms = {

    'Training' : transforms.Compose([
        transforms.RandomResizedCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'Testing': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()

    ])
}

## Loading Data

In [7]:
# directory: where training and testing data are
base_path = os.getcwd()
data_dir = os.path.join(base_path, 'Datasets/TumorDataset')

### START CODE HERE

# datasets.ImageFolder: (https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html)
# torch.utils.data.DataLoader: (https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

# image_datasets are dictionary of (type of dataset, dataloader)
# type of dataset are training and testing
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in data_transforms}

# DataLoader helps us for better performance and experience in data loading
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in data_transforms}
### END CODE HERE

dataset_sizes = {x: len(image_datasets[x]) for x in None}
class_names = image_datasets['Training'].classes

dataset_sizes, class_names

TypeError: 'NoneType' object is not iterable

## Samples of data

In [ ]:
samples, labels = next(iter(dataloaders['Testing']))
plt.figure(figsize=(17, 10))
plt.axis('off')
for i in range(32):
    plt.subplot(4, 8, i+1)
    plt.imshow(samples[i].permute(1, 2, 0))
    plt.title(class_names[labels[i]])
    plt.axis('off')

# Model

## Loading Model

In [ ]:
# Loading are pretrained model in this task our model is resnet50 (https://www.youtube.com/watch?v=mGMpHyiN5lk)
### START CODE HERE

# Loading pretrained model
model = models.resnet50(pretrained=None)
for param in model.parameters():
    param.requires_grad = False
### END CODE HERE
model

## Preparing Model

In [ ]:
### START CODE HERE

# You have to change the (fc) layer of the model to compatible with your data
model.fc = nn.Linear(model.fc.in_features, None)

### END CODE HERE
model = model.to(device)
model

# Training

## Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()

## Optimizer

In [ ]:
# you have to change it for better performance
optimizer = optim.Adam(model.parameters(), lr=None)

## Others

In [ ]:
# you can have other thongs like learning rate scheduler and ...

## Train

In [ ]:
### START CODE HERE

losses = []
EPOCH = None

# for training part you have to set model to train mode
model.train()

# loop on epochs
for e in tqdm(range(EPOCH)):

  # loop on batches
  for inputs, labels in dataloaders[None]:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # set the grad to zero
    optimizer.None
    
    # forward part
    # hint: using of pytorch max method (https://pytorch.org/docs/stable/generated/torch.max.html)
    outputs = model(None)
    _, preds = None

    #  compute loss
    loss = criterion(None, None)
    
    # backward part
    loss.backward()

    # update parameters
    optimizer.step()

  # you have to append loss for each epoch
  losses.append(None)
### END CODE HERE

## Plot loss function

In [ ]:
# you have to calculate losses arrayin Train part
plt.plot(list(range(len(losses))), losses)
plt.show()

## Evaluate model

In [ ]:
### START CODE HERE

def calc_accuracy(data, model):
  corrects = 0

  # for testing part you have to set model to eval mode
  model.eval()
  for inputs, labels in tqdm(dataloaders[data]):
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      with torch.no_grad():
        outputs = model(None)
        _, preds = None
        corrects += torch.sum(preds == labels.data)
  return corrects.double() / dataset_sizes[data]

### END CODE HERE

In [ ]:
# accuracy of training data
calc_accuracy(None, None)

In [ ]:
# accuracy of testing data
calc_accuracy(None, None)

# Saving Model

In [ ]:
PATH = os.path.join(base_path, 'model.ci')
torch.save(model, PATH)

# Loading and eval Model

In [ ]:
### START CODE HERE

model_for_eval = torch.load(PATH)
model_for_eval.to(device)

### END CODE HERE

In [ ]:
model_for_eval

In [ ]:
# accuracy of training data by loadded model
calc_accuracy(None, None)

In [ ]:
# accuracy of testing data by loadded model
calc_accuracy(None, None)